In [68]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

T_c = 10
T_l = 12

# create state space and initial state probabilities

states = ['acqusition', 'pre-processing', 'transfer']
pi = [0.35, 0.35, 0.3] #random init
state_space = pd.Series(pi, index=states, name='states')
print(state_space)
print(state_space.sum())

acqusition        0.35
pre-processing    0.35
transfer          0.30
Name: states, dtype: float64
1.0


In [64]:
# create transition matrix
# equals transition probability matrix of changing states given a state
# matrix is size (M x M) where M is number of states

q_df = pd.DataFrame(columns=states, index=states)
q_df.loc[states[0]] = [0.4, 0.2, 0.4]
q_df.loc[states[1]] = [0.45, 0.45, 0.1]
q_df.loc[states[2]] = [0.45, 0.25, .3]

print(q_df)

q = q_df.values
print('\n', q, q.shape, '\n')
print(q_df.sum(axis=1))

               acqusition pre-processing transfer
acqusition            0.4            0.2      0.4
pre-processing       0.45           0.45      0.1
transfer             0.45           0.25      0.3

 [[0.4 0.2 0.4]
 [0.45 0.45 0.1]
 [0.45 0.25 0.3]] (3, 3) 

acqusition        1.0
pre-processing    1.0
transfer          1.0
dtype: float64


In [65]:
from pprint import pprint 

# create a function that maps transition probability dataframe 
# to markov edges and weights

def _get_markov_edges(Q):
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx,col)] = Q.loc[idx,col]
    return edges

edges_wts = _get_markov_edges(q_df)
pprint(edges_wts)

{('acqusition', 'acqusition'): 0.4,
 ('acqusition', 'pre-processing'): 0.2,
 ('acqusition', 'transfer'): 0.4,
 ('pre-processing', 'acqusition'): 0.45,
 ('pre-processing', 'pre-processing'): 0.45,
 ('pre-processing', 'transfer'): 0.1,
 ('transfer', 'acqusition'): 0.45,
 ('transfer', 'pre-processing'): 0.25,
 ('transfer', 'transfer'): 0.3}


In [66]:
# create graph object
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(states)
print(f'Nodes:\n{G.nodes()}\n')

# edges represent transition probabilities
for k, v in edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
print(f'Edges:')
pprint(G.edges(data=True))    

#pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot')
#nx.draw_networkx(G, pos)

# create edge labels for jupyter plot but is not necessary
#edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
#nx.draw_networkx_edge_labels(G , pos, edge_labels=edge_labels)
#nx.drawing.nx_pydot.write_dot(G, 'pet_dog_markov.dot')

Nodes:
['acqusition', 'pre-processing', 'transfer']

Edges:
OutMultiEdgeDataView([('acqusition', 'acqusition', {'weight': 0.4, 'label': 0.4}), ('acqusition', 'pre-processing', {'weight': 0.2, 'label': 0.2}), ('acqusition', 'transfer', {'weight': 0.4, 'label': 0.4}), ('pre-processing', 'acqusition', {'weight': 0.45, 'label': 0.45}), ('pre-processing', 'pre-processing', {'weight': 0.45, 'label': 0.45}), ('pre-processing', 'transfer', {'weight': 0.1, 'label': 0.1}), ('transfer', 'acqusition', {'weight': 0.45, 'label': 0.45}), ('transfer', 'pre-processing', {'weight': 0.25, 'label': 0.25}), ('transfer', 'transfer', {'weight': 0.3, 'label': 0.3})])


In [67]:
#hidden markov model
# create state space and initial state probabilities

hidden_states = ['in sync', 'out of sync']
pi = [0.5, 0.5]
state_space = pd.Series(pi, index=hidden_states, name='states')
print(state_space)
print('\n', state_space.sum())

in sync        0.5
out of sync    0.5
Name: states, dtype: float64

 1.0


In [55]:
# create hidden transition matrix
# a or alpha 
#   = transition probability matrix of changing states given a state
# matrix is size (M x M) where M is number of states

a_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
a_df.loc[hidden_states[0]] = [0.7, 0.3]
a_df.loc[hidden_states[1]] = [0.4, 0.6]

print(a_df)

a = a_df.values
print('\n', a, a.shape, '\n')
print(a_df.sum(axis=1))

            in sync out of sync
in sync         0.7         0.3
out of sync     0.4         0.6

 [[0.7 0.3]
 [0.4 0.6]] (2, 2) 

in sync        1.0
out of sync    1.0
dtype: float64


In [56]:
# create matrix of observation (emission) probabilities
# b or beta = observation probabilities given state
# matrix is size (M x O) where M is number of states 
# and O is number of different possible observations

observable_states = states

b_df = pd.DataFrame(columns=observable_states, index=hidden_states)
b_df.loc[hidden_states[0]] = [0.2, 0.6, 0.2]
b_df.loc[hidden_states[1]] = [0.4, 0.1, 0.5]

print(b_df)

b = b_df.values
print('\n', b, b.shape, '\n')
print(b_df.sum(axis=1))

            triggered sending not triggered yet
in sync           0.2     0.6               0.2
out of sync       0.4     0.1               0.5

 [[0.2 0.6 0.2]
 [0.4 0.1 0.5]] (2, 3) 

in sync        1.0
out of sync    1.0
dtype: float64


In [57]:
# create graph edges and weights

hide_edges_wts = _get_markov_edges(a_df)
pprint(hide_edges_wts)

emit_edges_wts = _get_markov_edges(b_df)
pprint(emit_edges_wts)

{('in sync', 'in sync'): 0.7,
 ('in sync', 'out of sync'): 0.3,
 ('out of sync', 'in sync'): 0.4,
 ('out of sync', 'out of sync'): 0.6}
{('in sync', 'not triggered yet'): 0.2,
 ('in sync', 'sending'): 0.6,
 ('in sync', 'triggered'): 0.2,
 ('out of sync', 'not triggered yet'): 0.5,
 ('out of sync', 'sending'): 0.1,
 ('out of sync', 'triggered'): 0.4}


In [59]:
# create graph object
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(hidden_states)
print(f'Nodes:\n{G.nodes()}\n')

# edges represent hidden probabilities
for k, v in hide_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)

# edges represent emission probabilities
for k, v in emit_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
    
print(f'Edges:')
pprint(G.edges(data=True))    
"""
pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='neato')
nx.draw_networkx(G, pos)

# create edge labels for jupyter plot but is not necessary
emit_edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G , pos, edge_labels=emit_edge_labels)
nx.drawing.nx_pydot.write_dot(G, 'pet_dog_hidden_markov.dot')
"""

Nodes:
['in sync', 'out of sync']

Edges:
OutMultiEdgeDataView([('in sync', 'in sync', {'weight': 0.7, 'label': 0.7}), ('in sync', 'out of sync', {'weight': 0.3, 'label': 0.3}), ('in sync', 'triggered', {'weight': 0.2, 'label': 0.2}), ('in sync', 'sending', {'weight': 0.6, 'label': 0.6}), ('in sync', 'not triggered yet', {'weight': 0.2, 'label': 0.2}), ('out of sync', 'in sync', {'weight': 0.4, 'label': 0.4}), ('out of sync', 'out of sync', {'weight': 0.6, 'label': 0.6}), ('out of sync', 'triggered', {'weight': 0.4, 'label': 0.4}), ('out of sync', 'sending', {'weight': 0.1, 'label': 0.1}), ('out of sync', 'not triggered yet', {'weight': 0.5, 'label': 0.5})])


"\npos = nx.drawing.nx_pydot.graphviz_layout(G, prog='neato')\nnx.draw_networkx(G, pos)\n\n# create edge labels for jupyter plot but is not necessary\nemit_edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}\nnx.draw_networkx_edge_labels(G , pos, edge_labels=emit_edge_labels)\nnx.drawing.nx_pydot.write_dot(G, 'pet_dog_hidden_markov.dot')\n"

In [60]:
# observation sequence of dog's behaviors
# observations are encoded numerically

obs_map = {'sleeping':0, 'eating':1, 'pooping':2}
obs = np.array([1,1,2,1,0,1,2,1,0,2,2,0,1,0,1])

inv_obs_map = dict((v,k) for k, v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in list(obs)]

print( pd.DataFrame(np.column_stack([obs, obs_seq]), 
                columns=['Obs_code', 'Obs_seq']) )

   Obs_code   Obs_seq
0         1    eating
1         1    eating
2         2   pooping
3         1    eating
4         0  sleeping
5         1    eating
6         2   pooping
7         1    eating
8         0  sleeping
9         2   pooping
10        2   pooping
11        0  sleeping
12        1    eating
13        0  sleeping
14        1    eating


In [61]:
# define Viterbi algorithm for shortest path
# code adapted from Stephen Marsland's, Machine Learning An Algorthmic Perspective, Vol. 2
# https://github.com/alexsosn/MarslandMLAlgo/blob/master/Ch16/HMM.py

def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
    
    # init blank path
    path = np.zeros(T)
    # delta --> highest probability of any path that reaches state i
    delta = np.zeros((nStates, T))
    # phi --> argmax by time step for each state
    phi = np.zeros((nStates, T))
    
    # init delta and phi 
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0

    print('\nStart Walk Forward\n')    
    # the forward algorithm extension
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            print('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # find optimal path
    print('-'*50)
    print('Start Backtrace\n')
    path[T-1] = np.argmax(delta[:, T-1])
    #p('init path\n    t={} path[{}-1]={}\n'.format(T-1, T, path[T-1]))
    for t in range(T-2, -1, -1):
        path[t] = phi[path[t+1], [t+1]]
        #p(' '*4 + 't={t}, path[{t}+1]={path}, [{t}+1]={i}'.format(t=t, path=path[t+1], i=[t+1]))
        print('path[{}] = {}'.format(t, path[t]))
        
    return path, delta, phi

path, delta, phi = viterbi(pi, a, b, obs)
print('\nsingle best state path: \n', path)
print('delta:\n', delta)
print('phi:\n', phi)


Start Walk Forward

s=0 and t=1: phi[0, 1] = 0.0
s=1 and t=1: phi[1, 1] = 0.0
s=0 and t=2: phi[0, 2] = 0.0
s=1 and t=2: phi[1, 2] = 0.0
s=0 and t=3: phi[0, 3] = 0.0
s=1 and t=3: phi[1, 3] = 1.0
s=0 and t=4: phi[0, 4] = 0.0
s=1 and t=4: phi[1, 4] = 0.0
s=0 and t=5: phi[0, 5] = 0.0
s=1 and t=5: phi[1, 5] = 1.0
s=0 and t=6: phi[0, 6] = 0.0
s=1 and t=6: phi[1, 6] = 0.0
s=0 and t=7: phi[0, 7] = 0.0
s=1 and t=7: phi[1, 7] = 1.0
s=0 and t=8: phi[0, 8] = 0.0
s=1 and t=8: phi[1, 8] = 0.0
s=0 and t=9: phi[0, 9] = 0.0
s=1 and t=9: phi[1, 9] = 1.0
s=0 and t=10: phi[0, 10] = 1.0
s=1 and t=10: phi[1, 10] = 1.0
s=0 and t=11: phi[0, 11] = 1.0
s=1 and t=11: phi[1, 11] = 1.0
s=0 and t=12: phi[0, 12] = 1.0
s=1 and t=12: phi[1, 12] = 1.0
s=0 and t=13: phi[0, 13] = 0.0
s=1 and t=13: phi[1, 13] = 0.0
s=0 and t=14: phi[0, 14] = 0.0
s=1 and t=14: phi[1, 14] = 1.0
--------------------------------------------------
Start Backtrace



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [62]:
state_map = {0:'healthy', 1:'sick'}
state_path = [state_map[v] for v in path]

(pd.DataFrame()
 .assign(Observation=obs_seq)
 .assign(Best_Path=state_path))

NameError: name 'path' is not defined